In [7]:
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid
from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.nn import GCNConv
from torch_geometric.explain.metric import fidelity,characterization_score,unfaithfulness
dataset = 'Cora'
path = 'data\Planetoid'
dataset = Planetoid(path, dataset)
data = dataset[0]



In [2]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [4]:
for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [5]:
explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
)

In [6]:
node_index = 10
explanation = explainer(data.x, data.edge_index, index=node_index)
print(f'Generated explanations in {explanation.available_explanations}')

path = 'feature_importance.png'
explanation.visualize_feature_importance(path, top_k=10)
print(f"Feature importance plot has been saved to '{path}'")

path = 'subgraph.pdf'
explanation.visualize_graph(path)
print(f"Subgraph visualization plot has been saved to '{path}'")

Generated explanations in ['node_mask', 'edge_mask']
Feature importance plot has been saved to 'feature_importance.png'
Subgraph visualization plot has been saved to 'subgraph.pdf'


In [8]:

print(f'Generated explanations in {explanation.available_explanations}')

fid_pm = fidelity(explainer, explanation)
print("Fidelity:", fid_pm)
epsilon=0.99
# x =torch.tensor([0,0.2,0.4,0.6, 0.8 , 1])
if fid_pm[1]==1:
    char_score = characterization_score(fid_pm[0], epsilon)
else:
    char_score = characterization_score(fid_pm[0], fid_pm[1])
print("Characterization score:", char_score)

Generated explanations in ['node_mask', 'edge_mask']
Fidelity: (0.0, 0.0)


ZeroDivisionError: float division by zero

In [9]:
unfaithfulness1=unfaithfulness(explainer,explanation,top_k=10)

In [11]:
print(unfaithfulness1)

0.22430866956710815
